In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

os.listdir("/kaggle/input")


In [ ]:
import os

os.path.exists("/kaggle/input/ava-aesthetic-visual-assessment")


In [ ]:
os.listdir("/kaggle/input/ava-aesthetic-visual-assessment")


In [ ]:
os.listdir("/kaggle/input/ava-aesthetic-visual-assessment")



In [ ]:
import os

base = "/kaggle/input/ava-aesthetic-visual-assessment"
os.listdir(base)


In [ ]:
for item in os.listdir(base):
    path = os.path.join(base, item)
    print(item, "-> directory" if os.path.isdir(path) else "-> file")


In [ ]:
img_dir = "/kaggle/input/ava-aesthetic-visual-assessment/images"
os.listdir(img_dir)[:5]


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

img_dir = "/kaggle/input/ava-aesthetic-visual-assessment/images"
files = os.listdir(img_dir)

img = Image.open(os.path.join(img_dir, files[0]))

plt.imshow(img)
plt.axis("off")


In [ ]:
!nvidia-smi


In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# Paths
DATA_ROOT = "/kaggle/input/ava-aesthetic-visual-assessment"
IMG_DIR = f"{DATA_ROOT}/images"
CSV_PATH = f"{DATA_ROOT}/ground_truth_dataset.csv"


In [ ]:
# Load the AVA CSV
df = pd.read_csv(CSV_PATH)
print(df.head())
print(df.columns)  # check columns to see 'image_num', 'vote_1'..'vote_10'


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # ignore truncated images

class AVADataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        self.vote_cols = [f"vote_{i}" for i in range(1, 11)]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = str(int(self.df.iloc[idx]['image_num'])) + ".jpg"
        img_path = os.path.join(self.img_dir, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            new_idx = np.random.randint(0, len(self.df))
            return self.__getitem__(new_idx)

        votes = self.df.iloc[idx][self.vote_cols].values.astype(float)
        total_votes = votes.sum()
        mean_score = sum((i+1)*votes[i] for i in range(10)) / total_votes

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(mean_score, dtype=torch.float32)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:
subset_size = 20000  # pick 20k images for fast demo
df_subset = df.sample(n=subset_size, random_state=42).reset_index(drop=True)

dataset = AVADataset(df_subset, IMG_DIR, transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)


In [ ]:
class AestheticModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.fc = nn.Linear(512, 1)

    def forward(self, x):
        return self.backbone(x)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AestheticModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
EPOCHS = 1 # fast demo

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for imgs, scores in loader:
        imgs = imgs.to(device)
        scores = scores.to(device).unsqueeze(1)

        preds = model(imgs)
        loss = criterion(preds, scores)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} | Loss: {running_loss/len(loader):.4f}")


In [ ]:
torch.save(model.state_dict(), "aesthetic_model.pth")
print("Aesthetic scoring model saved!")


In [ ]:
!pip install diffusers transformers accelerate safetensors --quiet

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

In [ ]:
prompt = "a realistic portrait of a smiling woman, studio lighting"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "A calm beach during sunset, waves gently crashing, ultra-detailed"
image = pipe(prompt).images[0]
image

In [ ]:
prompt = "a beautiful landscape with mountains and lake, realistic lighting"
image = pipe(prompt).images[0]
image


In [ ]:
model.eval()  # ensure model is in eval mode

def aesthetic_score(image):
    img_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        score = model(img_tensor)
    return score.item()

score = aesthetic_score(image)
print("Aesthetic Score:", score)

In [ ]:
# -------------------------------
# EVALUATION USING MSE
# -------------------------------

model.eval()  # switch to evaluation mode

total_mse = 0.0
num_batches = 0

with torch.no_grad():  # no gradients during evaluation
    for imgs, scores in loader:
        imgs = imgs.to(device)
        scores = scores.to(device).unsqueeze(1)

        preds = model(imgs)
        mse = criterion(preds, scores)

        total_mse += mse.item()
        num_batches += 1

final_mse = total_mse / num_batches

print(f"Final Evaluation MSE: {final_mse:.4f}")
